# Import libraries

In [5]:
from __future__ import unicode_literals

path_direction = "resources/transition-data.txt"

In [6]:
import random
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm

data = [x.strip() for x in tqdm.tqdm(codecs.open(F'{path_direction}','rU','utf-8').readlines())]

100%|██████████| 1/1 [00:00<?, ?it/s]


In [7]:
print(data)

['ما از تهران با پیکان جوانان گوجه ای به مشهَدالرضا رفتیم.']


# Normalization

In [8]:
from hazm import *

normalizer = Normalizer()

# data_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(datas)]
data_normalized = [normalizer.normalize(y) for y in data]

In [9]:
for x in data_normalized :
    print(''.join(x))

ما از تهران با پیکان جوانان گوجه‌ای به مشهدالرضا رفتیم.


# Sentence tokenizing

In [10]:
data_sentences = [sent_tokenize(''.join(x)) for x in tqdm.tqdm(data_normalized)]

100%|██████████| 1/1 [00:00<00:00, 1088.86it/s]


# Find Spans

In [11]:
spans = []
for i in range(len(data_sentences)):
    start = 1
    end = 1
    current_spans = []
    for j in data_sentences[i][0]:
        end += 1
        if j == " " or j == ".":
            current_spans.append([start, end-2])
            start = end
    spans.append(current_spans)
print(spans)      

[[[1, 2], [4, 5], [7, 11], [13, 14], [16, 20], [22, 27], [29, 35], [37, 38], [40, 48], [50, 54]]]


# Word tokenizing

In [12]:
for x in data_sentences:
    print('###'.join(x))

ما از تهران با پیکان جوانان گوجه‌ای به مشهدالرضا رفتیم.


In [13]:
data_tokens = [[word_tokenize(sentence) for sentence in sentences] for sentences in tqdm.tqdm(data_sentences)]

100%|██████████| 1/1 [00:00<00:00,  4.42it/s]


In [14]:
data_tokens

[[['ما',
   'از',
   'تهران',
   'با',
   'پیکان',
   'جوانان',
   'گوجه\u200cای',
   'به',
   'مشهدالرضا',
   'رفتیم',
   '.']]]

# Utility functions

In [15]:
def read_database(file_path):
    file = open(file_path, "r", encoding="utf-8")
    return file.read()

def read_list_from_file(file_path):
    lst = read_database(file_path)
    lst = lst.strip('][').split(', ')
    lst = lst[0].replace("\"", "").replace(",", "").replace(" ", "").split("\n")
    final_list = []
    for string in lst:
        if (string != ""):
            final_list.append(string)
    return final_list

# Read databases of words

In [16]:
cars = read_list_from_file("database/cars.txt")
vehicles = read_list_from_file("database/vehicle.txt")
vehicles = cars + vehicles
vehicles = [normalizer.normalize(y) for y in vehicles]
#########
countries = read_list_from_file("database/countries.txt")
address_keyword = read_list_from_file("database/address keyword.txt")
cities = read_list_from_file("database/cities.txt")
provincies = read_list_from_file("database/provincies.txt")
places = read_list_from_file("database/places.txt")
places = countries + address_keyword + provincies + places + cities
places = [normalizer.normalize(y) for y in places]

# Main algorithm

In [18]:
result = []
for j in range(len(data_tokens)):
    sentence = data_tokens[j]
    sentence_full = data[j]
    sentence = sentence[0]
    res_dict = {}
    ###
    res_dict["from"] = ""
    res_dict["from_span"] = "[-1, -1]"
    res_dict["to"] = ""
    res_dict["to_span"] = "[-1, -1]"
    res_dict["vehicle"] = ""
    res_dict["vehicle_span"] = "[-1, -1]"
    ###
    for i in range(len(sentence)-1):
        if sentence[i] == "از" or sentence[i] == "مبدا":
            if sentence[i+1] in places:
                res_dict["from"] = sentence[i+1]
                res_dict["from_span"] = spans[j][i]
        if sentence[i] == "به" or sentence[i] == "مقصد":
            if sentence[i+1] in places:
                res_dict["to"] = sentence[i+1]
                res_dict["to_span"] = spans[j][i]
        if sentence[i] == "با" or sentence[i] == "خودروی" or sentence[i] == "ماشین":
            if sentence[i+1] in vehicles:
                res_dict["vehicle"] = sentence[i+1]
                res_dict["vehicle_span"] = spans[j][i]
    result.append(res_dict)

In [19]:
result

[{'from': 'تهران',
  'from_span': [4, 5],
  'to': 'مشهدالرضا',
  'to_span': [37, 38],
  'vehicle': 'پیکان',
  'vehicle_span': [13, 14]}]